In [1]:
! pip install transformers datasets evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 35.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.9 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installe

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
data = pd.read_csv('/content/drive/MyDrive/multitude_split/dataset_all.csv')
data.head()

,text,label,multi_label,split,language,length,source,word_count,unique_word_count,char_count,...,question_mark_count,exclamation_mark_count,flesch_reading_ease,gunning_fog_index,first_person_pronoun_count,person_entity_count,date_entity_count,uniqueness_bigram,uniqueness_trigram,syntax_variety
0,Der Ausbruch des Coronavirus hat die Entwicklu...,1,text-davinci-003,test,de,174,MULTITuDE_MassiveSumm_spiegel,199.0,118.0,1067.0,...,0.0,0.0,-272.022170,11.156030,0.0,0.0,0.0,0.904040,0.979695,12.0
1,Alex Azar was officially sworn in as the U.S. ...,1,text-davinci-003,train,en,57,MULTITuDE_MassiveSumm_voanews,70.0,54.0,311.0,...,0.0,1.0,-186.793214,8.714286,0.0,5.0,2.0,1.000000,1.000000,11.0
2,Європейський союз вимагає зупинити розтрату ко...,1,gpt-3.5-turbo,test,uk,105,MULTITuDE_MassiveSumm_interfax,130.0,82.0,691.0,...,0.0,0.0,-269.236538,11.015385,0.0,0.0,0.0,0.860465,0.929688,14.0
3,"Yesterday, hundreds of Zambian university stud...",1,text-davinci-003,train,en,254,MULTITuDE_MassiveSumm_voanews,292.0,149.0,1419.0,...,0.0,0.0,-231.229869,11.440100,1.0,1.0,1.0,0.876289,0.965517,13.0
4,"In a narrow and highly watched vote, the US Se...",1,gpt-4,train,en,416,MULTITuDE_MassiveSumm_voanews,476.0,242.0,2259.0,...,0.0,0.0,-224.855788,13.160504,1.0,2.0,2.0,0.871579,0.974684,15.0


In [4]:
from transformers import AutoTokenizer

# tokenizer = AutoTokenizer.from_pretrained("microsoft/mdeberta-v3-base")
tokenizer = AutoTokenizer.from_pretrained("intfloat/multilingual-e5-large")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

In [5]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, max_length=300)

In [6]:
data_test = data[data["split"] == "test"]
data_val = data[data["split"] == "test"]
# select 100 samples from each language following same label distribution
data_val = data_val.groupby("language").apply(lambda x: x.sample(100)).reset_index(drop=True)
data_train = data[data["split"] == "train"]
#print no. of each label for each language
print(data_train.groupby(["language", "label"]).size())
print(data_test.groupby(["language", "label"]).size())
print(data_val.groupby(["language", "label"]).size())

data_test = data_test[["text", "label"]]
data_train = data_train[["text", "label"]]
data_val = data_val[["text", "label"]]


language  label
en        0         2820
          1        24149
es        0          937
          1         7973
ru        0          999
          1         7908
dtype: int64
language  label
ar        0         299
          1        2374
ca        0         300
          1        2391
cs        0         300
          1        2389
de        0         292
          1        2393
en        0         277
          1        2214
es        0         284
          1        2392
nl        0         299
          1        2396
pt        0         287
          1        2386
ru        0         300
          1        2371
uk        0         298
          1        2370
zh        0         300
          1        2383
dtype: int64
language  label
ar        0         9
          1        91
ca        0         3
          1        97
cs        0         5
          1        95
de        0         9
          1        91
en        0        12
          1        88
es        0        15
      

In [7]:
import datasets

data_train = datasets.Dataset.from_pandas(data_train)
data_test = datasets.Dataset.from_pandas(data_test)
data_val = datasets.Dataset.from_pandas(data_val)

tokenized_data_train = data_train.map(preprocess_function, batched=True)
tokenized_data_test = data_test.map(preprocess_function, batched=True)
tokenized_data_val = data_val.map(preprocess_function, batched=True)

Map:   0%|          | 0/44786 [00:00<?, ? examples/s]

Map:   0%|          | 0/29295 [00:00<?, ? examples/s]

Map:   0%|          | 0/1100 [00:00<?, ? examples/s]

In [8]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

In [9]:
import evaluate

accuracy = evaluate.load("accuracy")
f1 = evaluate.load("f1")
recall = evaluate.load("recall")
precision = evaluate.load("precision")

In [10]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [12]:
from transformers import TFAutoModelForSequenceClassification, TrainingArguments, Trainer

model = TFAutoModelForSequenceClassification.from_pretrained(
    "intfloat/multilingual-e5-large", num_labels=2)

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFXLMRobertaForSequenceClassification: ['embeddings.position_ids']
- This IS expected if you are initializing TFXLMRobertaForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFXLMRobertaForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFXLMRobertaForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for pred

In [13]:
# print the model summary
model.summary()

Model: "tfxlm_roberta_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 roberta (TFXLMRobertaMainL  multiple                  558840832 
 ayer)                                                           
                                                                 
 classifier (TFXLMRobertaCl  multiple                  1051650   
 assificationHead)                                               
                                                                 
Total params: 559892482 (2.09 GB)
Trainable params: 559892482 (2.09 GB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [14]:
# print details of all layers
for layer in model.layers:
    print(layer.__class__.__name__)  # Print the class name of the layer

    print(layer.name)

TFXLMRobertaMainLayer
roberta
TFXLMRobertaClassificationHead
classifier


In [15]:
from transformers import create_optimizer
import tensorflow as tf

batch_size = 8
num_epochs = 3
batches_per_epoch = len(tokenized_data_train) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)
optimizer, schedule = create_optimizer(init_lr=2e-5, num_warmup_steps=0, num_train_steps=total_train_steps)

In [16]:
tf_train_set = model.prepare_tf_dataset(
    tokenized_data_train,
    shuffle=True,
    batch_size=8,
    collate_fn=data_collator,
)

tf_validation_set = model.prepare_tf_dataset(
    tokenized_data_val,
    shuffle=False,
    batch_size=8,
    collate_fn=data_collator,
)

In [17]:
model.compile(optimizer=optimizer)

In [18]:
from transformers.keras_callbacks import KerasMetricCallback

metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_validation_set)

In [19]:
callbacks = [metric_callback]

In [20]:
model.fit(x=tf_train_set, validation_data=tf_validation_set, epochs=3, callbacks=callbacks)

Epoch 1/3


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
5598/5598 [==============================] - 5132s 892ms/step - loss: 0.3439 - val_loss: 0.3109 - accuracy: 0.9064
Epoch 2/3
5598/5598 [==============================] - 4925s 880ms/step - loss: 0.3410 - val_loss: 0.3109 - accuracy: 0.9064
Epoch 3/3
5598/5598 [==============================] - 4928s 880ms/step - loss: 0.3397 - val_loss: 0.3109 - accuracy: 0.9064


In [37]:
# prompt: save the model

model.save("/content/drive/MyDrive/multitude_split/e5")


In [ ]:
# # prompt: load the saved model
# from transformers import TFAutoModelForSequenceClassification, TrainingArguments, Trainer

# model = TFAutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/multitude_split/ne1")


All model checkpoint layers were used when initializing TFDebertaV2ForSequenceClassification.

All the layers of TFDebertaV2ForSequenceClassification were initialized from the model checkpoint at /content/drive/MyDrive/multitude_split/ne1.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDebertaV2ForSequenceClassification for predictions without further training.


In [35]:
model.summary()

Model: "tfxlm_roberta_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 roberta (TFXLMRobertaMainL  multiple                  558840832 
 ayer)                                                           
                                                                 
 classifier (TFXLMRobertaCl  multiple                  1051650   
 assificationHead)                                               
                                                                 
Total params: 559892482 (2.09 GB)
Trainable params: 559892482 (2.09 GB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [36]:
# # print architecture of each layer
# for layer in model.layers:
#     print(layer.get_config())
#     print(layer.name)

In [38]:
tf_test_set = model.prepare_tf_dataset(
    tokenized_data_test,
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)
predictions = model.predict(tf_test_set)

1831/1831 [==============================] - 1025s 560ms/step


In [39]:
predictions = np.argmax(predictions.logits, axis=-1)
true_labels = data_test["label"]

In [ ]:
predictions = np.argmax(predictions.logits, axis=-1)

In [ ]:
predictions

In [ ]:
true_labels = data_test["label"]

In [40]:
from sklearn.metrics import classification_report

print(classification_report(true_labels, predictions))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00      3236
           1       0.89      1.00      0.94     26059

    accuracy                           0.89     29295
   macro avg       0.44      0.50      0.47     29295
weighted avg       0.79      0.89      0.84     29295



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [28]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(true_labels, predictions)

In [29]:
# roc_auc score
from sklearn.metrics import roc_auc_score

roc_auc_score(true_labels, predictions)

0.5

In [30]:
TN = cm[0, 0]
FP = cm[0, 1]
FPR = FP / (FP + TN)

In [31]:
FPR

1.0

In [32]:
FN = cm[1, 0]
TP = cm[1, 1]
FNR = FN / (FN + TP)
FNR

0.0

0.0

In [ ]:
model